# Dealing with errors after a run

In this example, we run the model on a list of three glaciers:
two of them will end with errors: one because it already failed at
preprocessing (i.e. prior to this run), and one during the run. We show how to recover from these errors and analyze them.

## The run

In [1]:
# Locals
import oggm.cfg as cfg
from oggm import utils, workflow, tasks

# Initialize OGGM and set up the default run parameters
cfg.initialize(logging_level='WARNING')

# Here we override some of the default parameters
# How many grid points around the glacier?
# We make it small because we want the model to error because
# of flowing out of the domain
cfg.PARAMS['border'] = 80

# This is the important bit!
# We tell OGGM to continue despite of errors
cfg.PARAMS['continue_on_error'] = True

# Local working directory (where OGGM will write its output)
WORKING_DIR = utils.gettempdir('OGGM_Errors')
utils.mkdir(WORKING_DIR, reset=True)
cfg.PATHS['working_dir'] = WORKING_DIR

rgi_ids = ['RGI60-11.00897', 'RGI60-11.01450', 'RGI60-11.03295']

# Go - get the pre-processed glacier directories
gdirs = workflow.init_glacier_directories(rgi_ids, from_prepro_level=4)

# We can step directly to the experiment!
# Random climate representative for the recent climate (1985-2015)
# with a negative bias added to the random temperature series
workflow.execute_entity_task(tasks.run_random_climate, gdirs,
                             nyears=150, seed=0,
                             temperature_bias=-1)

# Write the compiled output
utils.compile_glacier_statistics(gdirs);
utils.compile_run_output(gdirs);

2021-02-02 22:46:57: oggm.cfg: Reading default parameters from the OGGM `params.cfg` configuration file.
2021-02-02 22:46:57: oggm.cfg: Multiprocessing switched ON according to the parameter file.
2021-02-02 22:46:57: oggm.cfg: Multiprocessing: using all available processors (N=8)
2021-02-02 22:46:57: oggm.cfg: PARAMS['border'] changed from `20` to `80`.
2021-02-02 22:46:57: oggm.cfg: PARAMS['continue_on_error'] changed from `False` to `True`.
2021-02-02 22:46:57: oggm.workflow: init_glacier_directories from prepro level 4 on 3 glaciers.
2021-02-02 22:46:57: oggm.workflow: Execute entity task gdir_from_prepro on 3 glaciers
2021-02-02 22:46:58: oggm.workflow: Execute entity task run_random_climate on 3 glaciers
2021-02-02 22:46:58: oggm.core.flowline: InvalidWorkflowError occurred during task run_random_climate on RGI60-11.03295: Need a valid `model_flowlines` file. If you explicitly want to use `inversion_flowlines`, set use_inversion_flowlines=True.
2021-02-02 22:47:04: oggm.core.flow

## Run output

In [2]:
# TODO

## What's next?

- return to the [OGGM documentation](https://docs.oggm.org)
- back to the [table of contents](welcome.ipynb)